In [3]:
# import packages
import pandas as pd
import altair as alt
import os

In [2]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [4]:
os.chdir('/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/unemployment')

In [7]:
os.getcwd()

'/Users/sambickel-barlow/RADataHub/RADataHub/ChartOfTheDay/unemployment'

In [195]:
emp_stats_age = pd.read_excel('a05sasep2025.xls', sheet_name='People', skiprows=3)

## Filling Values Horizontally (Left to Right)

There are several methods to fill values sideways in a DataFrame:

In [196]:
# Method 1: Forward fill (ffill) along axis=1 (columns)
# This fills NaN values with the last valid value to the left
emp_stats_age = emp_stats_age.fillna(method='ffill', axis=1)

/var/folders/qz/pj0lh7817m3c9ydwgfrtbtf00000gn/T/ipykernel_33740/1154285243.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  emp_stats_age = emp_stats_age.fillna(method='ffill', axis=1)


In [197]:
# Concatenate values from rows 1 and 2 with a space between them and set as column names
emp_stats_age.columns = emp_stats_age.iloc[0].astype(str) + '_' + emp_stats_age.iloc[1].astype(str) + ' ' + emp_stats_age.iloc[2].astype(str)
# Drop the original rows 1 and 2 since they're now used as column names
emp_stats_age = emp_stats_age.drop([emp_stats_age.index[1], emp_stats_age.index[2]])


In [198]:
emp_stats_age.reset_index(drop=True, inplace=True)
# Get all rows except index 1 and 2

keep_rows = list(range(4, len(emp_stats_age), 3))

emp_stats_age_wide = emp_stats_age.iloc[keep_rows].reset_index(drop=True)

In [199]:
emp_stats_age_long = emp_stats_age_wide.melt(id_vars=['nan_nan nan'], var_name='statistic', value_name='Value')
emp_stats_age_long.rename(columns={'nan_nan nan': 'Quarter'}, inplace=True)
emp_stats_age_long['Age_Group'] = emp_stats_age_long['statistic'].str.split('_').str[0]
emp_stats_age_long['Measure'] = emp_stats_age_long['statistic'].str.split('_').str[1]

# Filter out rows with non-date Quarter values (notes, etc.)
emp_stats_age_long = emp_stats_age_long[emp_stats_age_long['Quarter'].str.contains(r'\d{4}', na=False)]

emp_stats_age_long['Quarter'] = emp_stats_age_long['Quarter'].str.replace('Jan-Mar', 'Q1')
emp_stats_age_long['Quarter'] = emp_stats_age_long['Quarter'].str.replace('Apr-Jun', 'Q2')
emp_stats_age_long['Quarter'] = emp_stats_age_long['Quarter'].str.replace('Jul-Sep', 'Q3')
emp_stats_age_long['Quarter'] = emp_stats_age_long['Quarter'].str.replace('Oct-Dec', 'Q4')

emp_stats_age_long['Quarter'] = emp_stats_age_long['Quarter'].str[3:] + ' ' + emp_stats_age_long['Quarter'].str[:2]

# Fix the date parsing - reformat to YYYY-QX format that pandas understands
emp_stats_age_long['Quarter_formatted'] = emp_stats_age_long['Quarter'].str.replace(' ', '-')
emp_stats_age_long['Date'] = pd.PeriodIndex(emp_stats_age_long['Quarter_formatted'], freq='Q').to_timestamp(how='end')

In [200]:
emp_stats_age_long = emp_stats_age_long[emp_stats_age_long['Age_Group'] != 'Aged 16 and over']
emp_stats_age_long = emp_stats_age_long[emp_stats_age_long['Age_Group'] != 'Aged 16-64 ']

emp_stats_age_long['Age_Group'] = emp_stats_age_long['Age_Group'].str.replace('Aged ', '')
emp_stats_age_long['Age_Group'] = emp_stats_age_long['Age_Group'].str.replace('Age ', '')

In [201]:
unemp_age_long = emp_stats_age_long[emp_stats_age_long['Measure'] == 'Unemployment rate (%)']
unemp_age_long['Value'] = pd.to_numeric(unemp_age_long['Value'], errors='coerce') / 100

/var/folders/qz/pj0lh7817m3c9ydwgfrtbtf00000gn/T/ipykernel_33740/2732649568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemp_age_long['Value'] = pd.to_numeric(unemp_age_long['Value'], errors='coerce') / 100


In [249]:
# Filter data to start from 2005 like the inactivity chart
unemp_age_long_filtered = unemp_age_long[unemp_age_long['Date'] >= pd.to_datetime('2005-06-30')]

# Find last point for each Age Group
last_points_unemp = unemp_age_long_filtered.loc[unemp_age_long_filtered.groupby('Age_Group')['Date'].idxmax()]

# Add manual adjustments for label positions to avoid overlap
last_points_unemp = last_points_unemp.copy()

# Manual adjustments for unemployment chart
age_group_adjustments_unemp = {
    '16-17': {'dy': 0, 'dx': 5},
    '18-24': {'dy': 5, 'dx': 5},
    '25-34': {'dy': -10, 'dx': 5},
    '35-49': {'dy': -5, 'dx': 5},
    '50-64': {'dy': 7, 'dx': 5},
    '65+': {'dy': 15, 'dx': 5}
}

# Build the line chart
line_unemp = alt.Chart(unemp_age_long_filtered).mark_line().encode(
    x=alt.X('Date:T', axis=alt.Axis(labelFontSize=14)),
    y=alt.Y('Value:Q', axis=alt.Axis(format='%', labelFontSize=14)),
    color=alt.Color('Age_Group:N', legend=None),
    tooltip=['Date:T', 'Age_Group:N', alt.Tooltip('Value:Q', format='.1%')]
)

# Create separate text charts for each age group with individual positioning
text_charts_unemp = []
for age_group, adjustments in age_group_adjustments_unemp.items():
    group_data = last_points_unemp[last_points_unemp['Age_Group'] == age_group]
    if not group_data.empty:
        text_chart = alt.Chart(group_data).mark_text(
            align='left',
            fontWeight='bold',
            fontSize=14,
            dx=adjustments['dx'],
            dy=adjustments['dy']
        ).encode(
            x=alt.X('Date:T'),
            y=alt.Y('Value:Q'),
            text='Age_Group',
            color=alt.Color('Age_Group:N', legend=None)
        )
        text_charts_unemp.append(text_chart)

# Combine all charts
combined_text_unemp = alt.layer(*text_charts_unemp) if text_charts_unemp else alt.Chart()

footnote1_unemp = alt.Chart(pd.DataFrame({'text': [
    "Note: Unemployment rate is the proportion of people in the labour force who are seeking work but unable to find it."
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=10,
    fontStyle='italic',
    dy=175,
    x=-15
).encode(
    text='text'
)

footnote2_unemp = alt.Chart(pd.DataFrame({'text': [
    "Sources: ONS: A05 SA: Employment, unemployment and economic inactivity by age group (seasonally adjusted)"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=10,
    fontStyle='italic',
    dy=175,
    x=-15
).encode(
    text='text'
)

# Combine and display
final_chart_unemp = (line_unemp + combined_text_unemp + footnote2_unemp).properties(
    title='Unemployment Rate by Age Group in the UK',
    width=450,
    height=300
).configure_title(
    fontSize=16,
    anchor='start',
    fontWeight='bold'
)

final_chart_unemp

alt.LayerChart(...)

In [251]:
# Save to png
final_chart_unemp.save('unemployment_by_Age.png', scale_factor=2)
# Save to json
final_chart_unemp.save('unemployment_by_Age.json', scale_factor=2)

As youth unemployment climbs, the youth job guarentee scheme proposed by Rachel Reeves seeks a solution: guaranteed oppertunities, whether jobs, education or apprenticeship, for young people out of work or education for 18+ months.

In [ ]:
inact_age_long = emp_stats_age_long[emp_stats_age_long['Measure'] == 'Inactivity rate (%)']
inact_age_long['Value'] = pd.to_numeric(inact_age_long['Value'], errors='coerce') / 100

In [ ]:
inact_age_long = inact_age_long[inact_age_long['Date'] >= pd.to_datetime('2005-06-30')]

In [242]:
# Find last point for each Age Group
last_points_inact = inact_age_long.loc[inact_age_long.groupby('Age_Group')['Date'].idxmax()]

# Add manual adjustments for label positions to avoid overlap
last_points_inact = last_points_inact.copy()

# Manual adjustments - modify these based on your chart
age_group_adjustments = {
    '16-17': {'dy': 0, 'dx': 5},
    '18-24': {'dy': 0, 'dx': 5},
    '25-34': {'dy': -5, 'dx': 5},
    '35-49': {'dy':5, 'dx': 5},
    '50-64': {'dy': 0, 'dx': 5},
    '65+': {'dy': 0, 'dx': 5}
}

# Build the line chart
line_inact = alt.Chart(inact_age_long).mark_line().encode(
    x=alt.X('Date:T', axis=alt.Axis(labelFontSize=14)),
    y=alt.Y('Value:Q', axis=alt.Axis(format='%', labelFontSize=14)),
    color=alt.Color('Age_Group:N', legend=None),
    tooltip=['Date:T', 'Age_Group:N', alt.Tooltip('Value:Q', format='.1%')]
)

# Create separate text charts for each age group with individual positioning
text_charts = []
for age_group, adjustments in age_group_adjustments.items():
    group_data = last_points_inact[last_points_inact['Age_Group'] == age_group]
    if not group_data.empty:
        text_chart = alt.Chart(group_data).mark_text(
            align='left',
            fontWeight='bold',
            fontSize=14,
            dx=adjustments['dx'],
            dy=adjustments['dy']
        ).encode(
            x=alt.X('Date:T'),
            y=alt.Y('Value:Q'),
            text='Age_Group',
            color=alt.Color('Age_Group:N', legend=None)
        )
        text_charts.append(text_chart)

# Combine all charts
combined_text = alt.layer(*text_charts) if text_charts else alt.Chart()

footnote1 = alt.Chart(pd.DataFrame({'text': [
    "Note: Inactivity rate is the proportion of people not in the labour force (neither working nor seeking work)."
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=10,
    fontStyle='italic',
    dy=175,
    x=-15
).encode(
    text='text'
)

footnote2 = alt.Chart(pd.DataFrame({'text': [
    "Sources: ONS: A05 SA: Employment, unemployment and economic inactivity by age group (seasonally adjusted)"
]})).mark_text(
    align='left',
    baseline='top',
    fontSize=10,
    fontStyle='italic',
    dy=190,
    x=-15
).encode(
    text='text'
)

# Combine and display
final_chart = (line_inact + combined_text + footnote1 + footnote2).properties(
    title='Inactivity Rate by Age Group in the UK',
    width=450,
    height=300
).configure_title(
    fontSize=16,
    anchor='start',
    fontWeight='bold'
)

final_chart

alt.LayerChart(...)

In [ ]:
# Save to png
final_chart.save('inactivity_by_Age.png', scale_factor=2)
# Save to json
final_chart.save('inactivity_by_Age.json', scale_factor=2)

In [ ]:
# The proposed Youth Jobs Guarantee would offer young people out of work or education for 18 months or more a guaranteed work placement. This proposal comes amid an upward creeping trend of inactivity amoung younger cohorts, while older cohorts inactivity creeps downwards.

As youth inactivity climbs and older worker participation rises, the proposed Youth Jobs Guarantee offers a solution: guaranteed work placements for young people out of work or education for 18+ months.